In [31]:
import os
import re
import sys
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch_geometric.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GAE, VGAE
from torch_geometric.data import Data
from tqdm.auto import tqdm
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer


sys.path.append("../..")

from utils.loss import ListNet
from utils.data_processing import get_process_data
from utils.eval import retrieval_normalized_dcg_all, retrieval_precision_all
from utils.toolbox import same_seeds, get_freer_gpu, show_settings, split_data

In [2]:
data_dict = get_process_data('IMDB')
doc_raw = data_dict['dataset']['documents']

vectorizer = TfidfVectorizer(max_df=1.0, min_df=10, stop_words="english")
doc_tfidf = vectorizer.fit_transform(doc_raw).todense()
doc_embedding = np.load('./docvec_IMDB_SBERT_768d.npy')

# Build word to index and index to word maps
word2index = vectorizer.vocabulary_
index2word = {}
for i in word2index:
    index2word[word2index[i]] = i

Reusing dataset imdb (/dhome/casimir0304/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


Generating document tfidf representation...
Document TFIDF dim:(50000, 25768)


In [3]:
# Delete word based on vocabulary
PATTERN = r"(?u)\b\w\w+\b"
vocab_set = set(word2index)
def doc_filter(doc_raw, vocab):
    doc = re.findall(PATTERN, doc_raw.lower())
    return [x for x in doc if x in vocab]
doc_list = [doc_filter(doc, vocab_set) for doc in tqdm(doc_raw, desc="Delete word from raw document:")]

Delete word from raw document::   0%|          | 0/50000 [00:00<?, ?it/s]

In [4]:
window_size = 5
windows = []

# Traverse Each Document & Move window on each of them
for doc in doc_list:
    length = len(doc)
    if length <= window_size:
        windows.append(doc)
    else:
        for i in range(length-window_size+1):
            window = doc[i: i+window_size]
            windows.append(window)

In [5]:
word_freq = {}
word_pair_count = {}
for window in tqdm(windows, desc='Calculate word pair: '):
    appeared = set()
    for i in range(len(window)):
        if window[i] not in appeared:
            if window[i] in word_freq:
                word_freq[window[i]] += 1
            else:
                word_freq[window[i]] = 1
            appeared.add(window[i])
        if i != 0:
            for j in range(0, i):
                word_i = window[i]
                word_i_id = word2index[word_i]
                word_j = window[j]
                word_j_id = word2index[word_j]
                if word_i_id == word_j_id:
                    continue
                word_pair_str = str(word_i_id) + ',' + str(word_j_id)
                if word_pair_str in word_pair_count:
                    word_pair_count[word_pair_str] += 1
                else:
                    word_pair_count[word_pair_str] = 1
                word_pair_str = str(word_j_id) + ',' + str(word_i_id)
                if word_pair_str in word_pair_count:
                    word_pair_count[word_pair_str] += 1
                else:
                    word_pair_count[word_pair_str] = 1


Calculate word pair:   0%|          | 0/5137347 [00:00<?, ?it/s]

In [6]:
from math import log
row = []
col = []
edge = []
weight = []

# pmi as weights

num_window = len(windows)

for key in tqdm(word_pair_count, desc='Construct Edge: '):
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_freq[index2word[i]]
    word_freq_j = word_freq[index2word[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(i)
    col.append(j)
    if count >= 10:
        edge.append([i, j])
        edge.append([j, i])
    weight.append(pmi)

print('# of Node: {}\n# of Edge: {}'.format(len(word2index), len(edge)))

Construct Edge:   0%|          | 0/16114622 [00:00<?, ?it/s]

# of Node: 25768
# of Edge: 1947480


In [7]:
def MythNet(y_pred, y_true, eps=1e-10):
    # (1) y_pred: the decoded vector. 
    #     ex: tfidf score of each word in certain document.
    # (2) y_true: the vector before encoded. 
    #     ex: same as above.
    # (3) eps: a small number to avoid error when computing log operation. 
    #     ex: log0 will cause error while log(0+eps) will not.

    y_pred = y_pred.clone()
    y_true = y_true.clone()

    y_pred = torch.sigmoid(y_pred) 
    y_pred = torch.nn.functional.normalize(y_pred, dim=1)
    # y_true = torch.nn.functional.softmax(y_true, dim=1) 
    y_true = torch.nn.functional.normalize(y_true, dim=1)
    pred = y_pred + eps
    pred_log = torch.log(pred)

    return torch.mean(torch.sum(-y_true * pred_log, dim=1))
    
class IDEDataset(Dataset):
    def __init__(self, doc_embedding, doc_tfidf):
        assert len(doc_embedding) == len(doc_tfidf)
        
        self.doc_embedding = torch.FloatTensor(doc_embedding)
        self.doc_tfidf = torch.FloatTensor(doc_tfidf)
    
    def __getitem__(self, idx):
        return self.doc_embedding[idx], self.doc_tfidf[idx]
    
    def __len__(self):
        return len(doc_embedding)

In [8]:
# Pytorch Geometric Package
class LinearEncoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.conv = GCNConv(input_dim, output_dim)
    def forward(self, x, edge_index):
        return self.conv(x, edge_index)

class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [9]:
class GNN(nn.Module):
    def __init__(self, vocab_size, output_dim):
        super().__init__()
        self.word_embedding = nn.Parameter(torch.randn(vocab_size, output_dim))
        self.ae = VGAE(VariationalGCNEncoder(output_dim, output_dim))
        
    def forward(self,edge_index):
        encoded = self.ae.encode(self.word_embedding, edge_index)
        loss = self.ae.recon_loss(encoded, edge_index)
        return loss
    
    def get_word_embedding(self):
        return self.word_embedding

In [10]:
class Decoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.decode = nn.Sequential(
            nn.Linear(input_dim, input_dim*2),
            nn.BatchNorm1d(input_dim*2),
            nn.Tanh(),
            nn.Linear(input_dim*2, input_dim*4),
            nn.BatchNorm1d(input_dim*4),
            nn.Tanh(),
            nn.Linear(input_dim*4, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return self.decode(x)       

In [48]:
device = f"{get_freer_gpu()}"
EPOCH = 500
batch_size = 128
lr= 1e-4
vocab_size = len(word2index)
feature_dim = 128
doc_embed_dim = 768
config = {
    "topk":[10,30,50]
}

Using cuda 0 for training...


In [49]:
dataset = IDEDataset(doc_embedding, doc_tfidf)
train_length = int(len(dataset)*0.8)
valid_length = int(len(dataset)*0.2)

full_loader = DataLoader(dataset, batch_size=128)
train_dataset, valid_dataset = random_split(dataset, lengths=[train_length, valid_length],generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [50]:
autoencoder = GNN(vocab_size, feature_dim).to(device)
decoder = Decoder(doc_embed_dim, feature_dim).to(device)
optimizer_ae = torch.optim.Adam(autoencoder.parameters(), lr = lr)
optimizer_de = torch.optim.Adam(decoder.parameters(), lr = lr)

edge_index = torch.tensor(edge, dtype=torch.long).to(device)
# node = torch.nn.Parameter(torch.tensor(torch.randn(25768, 8), dtype=torch.float)).to(device)
# graph = Data(x=node, edge_index=edge_index.t().contiguous()).to(device)

In [51]:
def evaluate_Decoder(decoder, autoencoder, data_loader):
    results = defaultdict(list)
    decoder.eval()
    autoencoder.eval()
    
    with torch.no_grad():
        # predict all data
        for batch, (doc_emb, doc_label) in enumerate(data_loader):
            doc_emb, doc_label= doc_emb.to(device), doc_label.to(device)  
            doc_label = torch.nn.functional.normalize(doc_label, dim=1)
            # pred = torch.nn.functional.normalize(decoder(doc_emb), dim=1)
            pred = torch.nn.functional.normalize(torch.sigmoid(torch.matmul(decoder(doc_emb), torch.transpose(autoencoder.get_word_embedding(), 0, 1))), dim=1)

            # Precision
            precision_scores = retrieval_precision_all(pred, doc_label, k=config["topk"])
            for k, v in precision_scores.items():
                results['precision@{}'.format(k)].append(v)

            # NDCG
            ndcg_scores = retrieval_normalized_dcg_all(pred, doc_label, k=config["topk"])
            for k, v in ndcg_scores.items():
                results['ndcg@{}'.format(k)].append(v)

        for k in results:
            results[k] = np.mean(results[k])

    return results

In [52]:
print(autoencoder.get_word_embedding())
for epoch in range(EPOCH):
    train_loss = 0
    autoencoder.train()
    decoder.train()
    update_ae = True
    for batch, (doc_emb, doc_label) in enumerate(tqdm(train_loader, desc='Training')):
        doc_emb, doc_label = doc_emb.to(device), doc_label.to(device)
        # decoded = decoder(doc_emb)
        decoded = torch.matmul(decoder(doc_emb), torch.transpose(autoencoder.get_word_embedding(), 0, 1))
        loss = MythNet(decoded, doc_label)
        if update_ae:
            update_ae = False
            ae_loss = autoencoder(edge_index.t().contiguous())
            loss += ae_loss
        loss.backward()
        
        optimizer_ae.step()
        optimizer_de.step()
        optimizer_ae.zero_grad()
        optimizer_de.zero_grad()
        
        train_loss += loss.item()
    print("[Epoch {:02d}]".format(epoch+1))
    res = evaluate_Decoder(decoder, autoencoder, valid_loader)
    for key,val in res.items():
        print(f"{key}:{val:.4f}")
print(autoencoder.get_word_embedding()) 

Parameter containing:
tensor([[-0.2307, -1.4462,  0.9185,  ...,  0.9183,  0.4511, -0.5038],
        [ 1.2987, -0.4412, -0.8500,  ..., -0.9998, -0.0060,  0.2104],
        [-1.2789,  0.2494, -0.5027,  ..., -1.1729,  1.0544,  0.1979],
        ...,
        [ 1.5124, -1.5924, -1.1022,  ..., -0.1607,  0.4827,  1.1926],
        [ 0.7896, -1.4215, -0.3270,  ..., -0.8665, -0.5517,  0.1632],
        [-0.2240,  1.8189, -1.2375,  ...,  0.3407, -0.1206,  1.2510]],
       device='cuda:0', requires_grad=True)


Training:   0%|          | 0/313 [00:00<?, ?it/s]

[Epoch 01]
precision@10:0.0035
precision@30:0.0044
precision@50:0.0050
ndcg@10:0.0014
ndcg@30:0.0021
ndcg@50:0.0027
ndcg@all:0.2765


Training:   0%|          | 0/313 [00:00<?, ?it/s]

[Epoch 02]
precision@10:0.0054
precision@30:0.0088
precision@50:0.0103
ndcg@10:0.0019
ndcg@30:0.0035
ndcg@50:0.0047
ndcg@all:0.2896


...

[Epoch 498]
precision@10:0.3616
precision@30:0.2137
precision@50:0.1690
ndcg@10:0.1885
ndcg@30:0.1574
ndcg@50:0.1504
ndcg@all:0.4321


Training:   0%|          | 0/313 [00:00<?, ?it/s]

[Epoch 499]
precision@10:0.3616
precision@30:0.2135
precision@50:0.1694
ndcg@10:0.1887
ndcg@30:0.1576
ndcg@50:0.1509
ndcg@all:0.4323


Training:   0%|          | 0/313 [00:00<?, ?it/s]

[Epoch 500]
precision@10:0.3604
precision@30:0.2129
precision@50:0.1687
ndcg@10:0.1886
ndcg@30:0.1575
ndcg@50:0.1506
ndcg@all:0.4322
Parameter containing:
tensor([[-0.3064, -1.6504,  0.7370,  ...,  0.3967,  0.5263, -0.5324],
        [ 1.3454, -1.0220, -0.5923,  ..., -1.4330, -0.4407,  0.1754],
        [-0.9299,  0.6226, -0.5324,  ..., -1.7264,  0.7399,  0.4072],
        ...,
        [ 1.0330, -1.8810, -0.7247,  ..., -0.0432,  0.4095,  1.2631],
        [ 1.0926, -1.5971, -0.2084,  ..., -1.4981, -0.5659, -0.0134],
        [-0.5627,  1.7876, -1.0276,  ...,  0.2475, -0.0991,  1.2782]],
       device='cuda:0', requires_grad=True)


In [ ]:
# Ignore the followings

In [9]:
class GNN_temp(nn.Module):
    def __init__(self, word_embedding, output_dim):
        super().__init__()
        self.word_embedding = word_embedding
        self.ae = VGAE(VariationalGCNEncoder(output_dim, output_dim))
        
    def forward(self,edge_index):
        encoded = self.ae.encode(self.word_embedding, edge_index)
        loss = self.ae.recon_loss(encoded, edge_index)
        return loss
    
    def get_word_embedding(self):
        return self.word_embedding

In [22]:
class Decoder_temp(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.word_embedding = nn.Parameter(torch.randn(hidden_dim, output_dim))
        self.ae = VGAE(VariationalGCNEncoder(hidden_dim, hidden_dim))
        self.transform = nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Sigmoid(),
        )
    def forward(self, x):
        docvec = self.transform(x)
        decoded = torch.sigmoid(torch.matmul(docvec, self.word_embedding))
        return decoded
 
    def get_ae_loss(self, edge_index):
        encoded = self.ae.encode(self.word_embedding, edge_index)
        loss = self.ae.recon_loss(encoded, edge_index)
        return loss

In [27]:
def evaluate_Decoder_temp(model, data_loader):
    results = defaultdict(list)
    model.eval()
        
    # predict all data
    for data in data_loader:
        doc_embs, target = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        
        target = torch.nn.functional.normalize(target, dim=1)
        pred = torch.nn.functional.normalize(torch.sigmoid(model(doc_embs)), dim=1)
    
        # Precision
        precision_scores = retrieval_precision_all(pred, target, k=config["topk"])
        for k, v in precision_scores.items():
            results['precision@{}'.format(k)].append(v)
        
        # NDCG
        ndcg_scores = retrieval_normalized_dcg_all(pred, target, k=config["topk"])
        for k, v in ndcg_scores.items():
            results['ndcg@{}'.format(k)].append(v)
        
    for k in results:
        results[k] = np.mean(results[k])

    return results